# Querying for Data in Excel

## Introduction

Excel is a wonderful introductory tool to the world of data science & analytics. While it is ultimately not as flexible or powerful as programming-centered tools such as Python, Excel is very good for general data exploration, analysis, and some data manipulation and reporting. Many times, it can be much quicker to find some data in an Excel sheet than it would be to find it using a Python script.

At the same time, it's also important to understand the limitations of Excel - once you start building multi-sheet files with numerous complex formulas and a couple of VBA macros, it's probably time to start thinking about moving on to more powerful tools.

In the following several cases, we will investigate how to use Excel for these particular use cases. For this case, we will learn how to do the basics of data exploration - finding the data we want and summarizing it so we can understand it better.

We will be working with the Excel workbook file [data/atp_matches_all_2020.xlsx](data/atp_matches_all_2020.xlsx) (**note**: you can find this Excel workbook in the `data/` folder of this case), which contains information for all tennis matches played during the 2020 season, as well as historical information about player rankings.


## It's rows and columns all the way

Excel's formatting is simple to understand at first glance - it is all composed of intersecting **rows** and **columns** , where each intersection between a row and a column is a **cell.** Rows are the _horizontal_ series of cells that are identified by a number, and columns are the _vertical_ series of cells identified by uppercase letters. In a single Excel file, you can find several **sheets** , each with its own set of rows and columns. Excel has a limitation of 1,048,576 rows and 16,384 columns per sheet, so keep this in mind in case you ever want to work with very large datasets.

It is also typical that the very first row of a sheet is special and is known as the **header row**, which gives names to the data contained in each column. Take a look at the values in the first row, and if you don't understand what some of them mean you can use the [data/matches_data_dictionary.txt](data/matches_data_dictionary.txt) file to find out. (**note**: you can find this text file in the `data/` folder of this case).

The [data/atp_matches_all_2020.xlsx](data/atp_matches_all_2020.xlsx) file has several sheets, each of them holding information for a particular set of tennis matches. Each column holds information about a match such as the player names, internal numeric IDs, the tournament where the match was played, the final score, etc.

## Filtering

**Filters** help us manually locate data we need without altering it. In order to create filters in any given sheet, follow these steps:

1. Select the whole table on which you want to filter. A good shortcut in case the only thing in your sheet is the table is to click the little arrow in between column A and row 1, which will select the whole sheet:

![](data/images/select_all.png)

2. In the Home tab of the Toolbar, click the "Sort & Filter" button. This will open a drop-down menu. Click on "Filter".

![](data/images/sort_and_filter.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; On the below image we see how to filter the Column K with header `winner_name` by a text filter `Daniel Elahi`. This will add a little arrow on the header row for all columns. This arrow allows you to filter the table to find exactly the data you need. 

![](data/images/header_filter.png)

### Exercise 1

How many matches did Daniel Elahi (the top-ranked Colombian tennis player) win in 2020? How many did he lose?

**Answer.**

Won 4 matches, lost 3

-------

### Exercise 2

Sheet `atp_matches_qual_chall_2020` has the data for Qualifying and Challenger matches played in 2020. How many matches did Novak Djokovic win and lose in the ATP main matches? How about the Qualifying & Challenger categories?

**Answer.**

ATP Main: won 41 matches, lost 5
ATP Qual: no matches

-------

## VLOOKUP & HLOOKUP

Filtering is one way to narrow down our results; however, the process of extracting the data we want is still very manual. How can we extract this data in a more automated way and use it elsewhere?

Enter the **`VLOOKUP`** and **`HLOOKUP`** functions. These names are short for "Vertical Lookup" and "Horizontal Lookup". Since our data is in vertical tables, we will use `VLOOKUP`. (`HLOOKUP` works the same as `VLOOKUP`, just with the axes flipped, so it should be a piece of cake after this.)

One example of something we can look up is the name of a player given his internal ID. The sheet `atp_rankings_current` has 4 populated columns and 4 columns we will fill later. The first has the date at which the ranking was calculated, the second has the player's ranking, the third has the player's internal ID, and the fourth has the amount of ATP points the player had at the date the rankings were calculated.

In Cell E2, we will determine the name of the #1 ranked player. The `VLOOKUP` function has 3 mandatory arguments and 1 optional argument:

1. The cell containing the value we will look up to help us find more information
2. The table in which to look for the value. This range of cells _must_ have the value we are looking for in its first column.
3. The number of the column where we can find the value we need, counting from the first column in our range.
4. Optionally, we can tell the function we want an exact match with FALSE in this argument.

Let's walk through each of the above for our specific use case:

1. Our value we want to look up is contained in cell D2.
2. For the range in which we need to look for the player name, we will use sheet `atp_matches_main_2020` and we will take the range of cells starting with H2 and ending with K1463. Note how we start our range in column H since it is the one that contains player IDs. We end it in column K because it contains player names, but we could end it in any other column after K as well in case we wanted data that is farther to the right.
3. Column K contains our desired data; since column H contains our lookup value, K is the 4th column.
4. We want an exact match, so we use FALSE as the last argument.

Thus, our final formula is:

```excel
VLOOKUP(C2, atp_matches_main_2020!$H$2:$K$1463, 4, FALSE)
```

We can see that Novak is currently the #1 rated player. If you extend the cell downwards, you will be able to see the names of the other ranked players as of April 2021 (e.g. #2 is Daniil Medvedev).

### Exercise 3

Who was the #2 player at the end of 2020? What is Daniel Elahi's current ranking?

**Hint:** The rankings for 2020 are in `atp_rankings_20s`.

**Answer.**

Rank 115

-------

Now we know how to use `VLOOKUP` and `HLOOKUP` to find information in a table. But it has some limitations - first, it can only look for data that is in a column that is after the initial column in a range. For example, it wouldn't be able to get data in columns A through G in our example. And second, it will return only the FIRST match in the range every time. So in case we want the data after the first match, we are out of luck. So VLOOKUP is best suited for tables where each row has a *unique identifier*, and that identifier is in the very first column. Neither of these is the case in our matches data.

## `INDEX` + `MATCH`

A more flexible (and all-around better) way to look up information is with the combination of the `INDEX` and `MATCH` functions. Using a combination of these, you can look into columns *before* the column with your initial value, look for values *beyond* the first that satisfy your set of conditions. The only downside is that these are a little harder to grasp at first, but we will get beyond that soon.

**`INDEX`** gives you the result of a value in a range given the range and the position of the value in that range. Take a look at the formula in B2 in the `aux_formulas` sheet. It looks at column F from row 9 to 28 (its first argument) and gets the fourth value (the second argument). It also works for horizontal ranges, and it even works for ranges with multiple columns and rows! The only difference in the latter case is that we must provide both a row and column number as arguments, in that order.

There is an example in cell B3, where we have all columns in addition to the same rows as before:

```excel
INDEX(B9:F28, 4, 3)
```

This enables us to find the value of a particular cell in a range given its position in that range. But how does this let us find something for which we do not know the position?

**`MATCH`** gets the position of any value in a horizontal or vertical range. It needs 3 arguments:

1. The value to look for
2. The horizontal or vertical range in which to look for it
3. (Optionally) The number 0 if you want to find an exact match, or 1 if an approximate match is enough. This is important! For most cases you probably want to use 0 for an exact match.

`MATCH` will return the position of your value in the list if it exists. There is an example in cell B4:

```excel
MATCH("Roger Federer", F9:F28, 0)
```

Now, how do we combine these 2 functions to do a full lookup? In place of the positional arguments of `INDEX`, we place `MATCH`. So, instead of telling `INDEX` in what row to look for, we let `MATCH` find the row for us. See the example in cell G4:

```excel
INDEX(B14:F33, MATCH(G3, F14:F33, 0))
```

We used `MATCH` to replace the first positional argument, and we told `INDEX` to use the value in the second column for the second positional argument. We could also use `MATCH` for the second positional argument so that it can tell us the column where we can find any particular characteristic. See cell G5 for an example.

### Exercise 4

Using `INDEX` + `MATCH`, find the ranking at the beginning of 2020 for all players in the current rankings table. Display these rankings in column F of the `atp_rankings_current` sheet. What was Daniel Elahi's ranking at the beginning of 2020?

**Answer.**

-------

Now we see why `INDEX` + `MATCH` is so powerful! Using `MATCH`, we can look for values in any row or column, without a care about the position of our lookup value. `INDEX` + `MATCH` is the gold standard for lookups in Excel, so keep it in mind and use it often.

## `INDIRECT` and `CHOOSE`

Although `INDEX` + `MATCH` will be your bread-and-butter for most lookup operations, there are a couple more useful formulas. **`INDIRECT`** is one of the simplest Excel functions to understand - it takes the name of a cell or a range and returns the value in that cell or range! The simplest example is `INDIRECT("A1")` - it will return the value in the A1 cell.

So what's the difference between that and just typing `=A1`? Well, truth be told the use cases are few and far between, but sometimes you have a need for the reference to that cell to never change, or you may want to use sheet names in a formula. Look at the example in sheet `aux_formulas`, cell B35:

```excel
INDEX(INDIRECT(B33&"!A2:F2059"),MATCH(B34,INDIRECT(B33&"!E2:E2059"),0),2)
```

We use the text in cell B33 together with a set range that we know holds values in several sheets, which lets us use the sheet names as part of a formula. Here, we duplicated the formula in cell G4, but now we can see the position of any given player in either ranking sheet.

### Exercise 5

Using `INDIRECT` along with `INDEX`, write a formula that returns the name of the loser in the first match in one of the 4 sheets containing match information.

**Answer.**

-------

Finally, **`CHOOSE`** lets you return a value at a given position in a list of values. It is somewhat like `INDEX`, but with one big difference - it cannot look inside ranges of cells, but rather only lists of values. This makes it somewhat limited in scope since most of the time you want to look inside ranges of values instead of a list, but it still has a few uses.

You can see an example in cells B44 and B45. The first formula works because the values are in a list, the second does not because `CHOOSE` cannot look into ranges. The values need not be single cells - they can be ranges as well.

In all honesty, 99.99% of the time you will use `INDEX`, but in complex cases where you want to select from a set of ranges, `CHOOSE` may come in handy.

## Summarizing data

Imagine we want to know how a player's age affects their chances of winning a match. Or we want to know how many aces were scored by losers in all matches, compared to how many aces were scored by winners. Or maybe we want to know the total points scored by a player over an entire season.

We can find the answers to these questions with the use of **aggregation functions** - functions which take several values and give us a single result. There are many such functions, but some of the most important are `SUM`, `SUMPRODUCT`, `AVERAGE`, `COUNT` and their choosy big brothers, `SUMIF` and `COUNTIF`.

### Aggregation functions

The **`SUM`** function is maybe the most used function out of them all. As its name implies, it lets you add a series of numbers easily. You can give it arguments to add in one of 2 ways: 1) as a list (similar to `CHOOSE`); or 2) as a range (similar to `INDEX`). Hence, the formula `SUM(A1,A2,A3,A4)` is identical to `SUM(A1:A4)`. Given that the range syntax is shorter, it is often preferred.

You can also add several ranges together. For example, `SUM(A1:A4, B7:F7)` will add all numbers in both ranges.

In cell B49 of the `aux_formulas` sheet, you can see the total number of aces scored across all the winning players in main matches over the 2020 season:

```excel
SUM(atp_matches_main_2020!AB2:AB1463)
```

Similarly, **`AVERAGE`** lets you find the result of adding all values in any given range and dividing the result by the number of summed values (i.e. the mean). In cell B50 of the `aux_formulas` sheet, we can find the average number of aces served by winners across all main matches.

### Exercise 6

Find the total and average number of aces scored across all losing players in main matches.

**Answer.**

-------

Next, we have **`SUMPRODUCT`**. `SUMPRODUCT` takes 2 ranges of the same size, takes the product of corresponding cells in each range (i.e. the product of the first cells in each range, the product of the second cells in each range, etc.), then sums up all of these products.

One of the most common uses of this function is for finding a weighted average. You are probably familiar with weighted averages from your grade school days - your grade was divided in several deliverables, each of which had a different weight, and your final grade was decided by the weighted average of all the deliverables.

Look at the example in cell F53 - we give the `SUMPRODUCT` function 2 ranges of equal size, which get multiplied and added up. In this particular case, we don't need to divide by anything because the weights range adds up to 1. But for other situations where this isn't the case, the weighted average needs to be divided by the sum of the weights. For example, see the formula in cell F65:

```excel
SUMPRODUCT(B64:B73,C64:C73)/SUM(B64:B73)
```

We can see that the weighted average is different from the normal average - don't confuse them!

### Exercise 7

Find the weighted average of the amount of winner serve points (`w_svpt`), with the minutes played in each match as the weight. How do you interpret the result?

**Answer.**

-------

Finally, we have the `COUNT` function, which counts number of cells in a given range that contain a number. It can be used to understand how much of a dataset is missing some characteristic. There is also the `COUNTA` function, which does exactly the same but count all non-empty cells, not only those with numbers.

For example, did you notice that not all matches have a minutes characteristic? With `COUNT`, we can find out how many of them have their minutes feature filled in. See the example in cells B76 to B78.


## Conditional Functions

Sometimes you will only want to average, sum, or count the values for a particular player, a particular country, or for any other particular *condition*. **Conditional functions** allow you to do just that - you provide them conditions, and they ignore any value along a range that does not fulfill that condition.

**`SUMIF`** and **`AVERAGEIF`** work similarly. They both take 2 mandatory arguments and 1 optional one:

1. The range on which to check the criteria or condition
2. The condition to check
3. (Optional) The range on which to sum or average. If this is not provided, then the operation is carried out on the range given in the first argument

For example, let's use `AVERAGEIF` to find the average duration of matches played where a Colombian player won. You can see the formula in cell B81 of the `aux_formulas` sheet:

```excel
AVERAGEIF(atp_matches_main_2020!N2:N1463,"=COL",atp_matches_main_2020!AA2:AA1463)
```

Conditions are given in double quotes. If you wanted to check if the values in a range are above 5, you would type ">5" as the condition. If you want to check if the values are less than or equal to 30, you would type "<=30". In the example above, we check the condition "=COL" along column N, which holds the country code for the winner of each match. We also gave the optional third argument with column AA, which holds the number of minutes in the match.

### Exercise 8

Find the average duration in minutes of matches played where a Norwegian player won (code NOR).

**Answer.**

-------

We can see that Norwegian players win matches almost 13 minutes faster than Colombian players do, which is associated with a more aggressive and explosive playing style.

**`SUMIF`** operates similarly - you can see an example in cell B82 of the `aux_formulas` sheet.

### Exercise 9

Find the sum of current ATP points for all top-50 players who are from Spain (code ESP).

**Hint:** Extract the country code for each top-50 player as an intermediate step in column G of `atp_rankings_current`.

**Answer.**

-------

**`COUNTIF`** is a little different. Since there are no numbers to sum or average, it just takes the first two arguments. Check out the example in cell B83 of the `aux_formulas` sheet.

### Exercise 10

Count how many players in the current top 50 are under 30 years old, and how many are at least 30 years old. What do you make of the results?

**Answer.**

-------

## Conclusions & Takeaways

In this case we learned how to look for information in Excel using a variety of methods:

1. Filters (to help us manually go to the data we need)
2. `VLOOKUP` and `HLOOKUP`
3. `INDEX` + `MATCH`
4. (Occasionally) `INDIRECT` or `CHOOSE`

We also learned about applying aggregation functions to our data in order to extract more meaning out if it (with functions such as `SUM`, `AVERAGE`, or `COUNT`).

In the following cases we will learn more about data transformation of text, using functions such as `CONCATENATE` or `RIGHT/LEFT`. And finally, we will see how we can better *summarize* and *visualize* our data with charts, pivot tables, and pivot charts.